# Modeling
## Estancia de investigación Maestría de Ciencia de Datos
## Aide Jazmín González Cruz

### 1. Carga de librerías

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from siuba import *
import time
from time import gmtime, strftime

import sys
import os
from os.path import dirname
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
sys.path.append(dirname('../src'))
from src.utils import utils


### 2. Carga de features y división de datos de entrenamiento y validación

In [3]:
df = utils.load_df("../Data/feature_selection.pkl")
df.head(5)

,ventana,max_altura,min_preglucosa,num_med_preglucosa,max_edad,min_edad,max_dif_vent_entre_consul,max_sistolica_a,avg_preglucosa,avg_edad,...,altura,comp_precoces_traumatismos,avg_enf_aparato_urinario,enf_circulacion_pulmonar_enf_corazon,malformaciones_congenitas,imc_r__x0_Peso normal,imc_r__x0_Sin dato,fiebre_y_enf_cardiacas_reumaticas,imc,label
0,1.0,1.64,108.000000,9.000000,49.0,47.0,100.0,130.0,150.000000,48.160000,...,1.0,1.0,0.9,0.0,1.0,0.0,0.0,1.0,1.0,NaN
1,2.0,1.64,109.000000,4.000000,50.0,48.0,69.0,130.0,158.000000,49.000000,...,1.0,1.0,0.9,0.0,1.0,0.0,0.0,1.0,1.0,NaN
2,3.0,1.64,253.000000,1.000000,51.0,49.0,69.0,130.0,253.000000,50.142857,...,1.0,1.0,0.9,0.0,1.0,0.0,0.0,1.0,1.0,NaN
3,4.0,1.64,80.000000,4.000000,52.0,50.0,112.0,160.0,155.750000,51.200000,...,1.0,1.0,0.9,0.0,1.0,0.0,0.0,1.0,1.0,NaN
4,1.0,1.76,125.188235,3.941176,33.0,32.0,542.0,100.0,157.012007,32.500000,...,1.0,1.0,0.9,0.0,1.0,0.0,0.0,1.0,1.0,NaN


In [4]:
X = df.loc[:, df.columns != 'label']
print(X.shape)
Y = df[["label"]]
print(Y.shape)

(308, 139)
(308, 1)


In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3)

### 3. Magic Loop

- Diccionario de algoritmos a aplicar

In [ ]:
algorithms_dict = {'tree': 'tree_grid_search',
                   'random_forest': 'rf_grid_search',
                   'logistic': 'logistic_grid_search'}

- Diccionario de configuraciones

In [ ]:
grid_search_dict = {
                        'tree_grid_search': {
                            'max_depth': [5, 10, 15],
                            'min_samples_leaf': [3, 5, 7]
                        },
                        'rf_grid_search': {
                            'n_estimators': [30, 50, 100],
                            'max_depth': [5, 10, 15],
                            'min_samples_leaf': [3, 5, 10]
                        },
                        'logistic_grid_search':{
                            'C':np.logspace(-3,3,7),
                            'penalty':['l2']
                        }
                    }

- Diccionario de estimadores

In [ ]:
from sklearn.tree import DecisionTreeClassifier
estimators_dict = {'tree': DecisionTreeClassifier(random_state=1111),
                   'random_forest': RandomForestClassifier(oob_score=True, random_state=2222),
                   'logistic': LogisticRegression(random_state=3333) 
                  } 